In [1]:
import email
import os
from pathlib import Path
import platform


In [2]:
# Message files end with a dot. This is not supported on Windows.
# Workaround for Windows:
# Add a prefix "\\?\" to the path to force the OS to read the files.
cwd = os.getcwd()
if platform.system() == "Windows":
    cwd = "\\\\?\\" + cwd
dir = cwd + os.path.normpath("/maildir/allen-p/sent_items")

for mf in os.listdir(dir):
    message_path = Path(dir + os.path.normpath("/") + mf)
    with open(message_path, "r") as mfp:
        m = email.message_from_file(mfp)


In [4]:
for k, v in m.raw_items():
    print(k, v)


Message-ID <2621811.1075858639545.JavaMail.evans@thyme>
Date Thu, 7 Jun 2001 06:49:01 -0700 (PDT)
From k..allen@enron.com
To stephanie.sever@enron.com
Subject 
Mime-Version 1.0
Content-Type text/plain; charset=us-ascii
Content-Transfer-Encoding 7bit
X-From Allen, Phillip K. </O=ENRON/OU=NA/CN=RECIPIENTS/CN=PALLEN>
X-To Sever, Stephanie </O=ENRON/OU=NA/CN=RECIPIENTS/CN=Ssever>
X-cc 
X-bcc 
X-Folder \PALLEN (Non-Privileged)\Allen, Phillip K.\Sent Items
X-Origin Allen-P
X-FileName PALLEN (Non-Privileged).pst
